**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N60eb3bd2d192496584fc05a87ee9e0ef (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [ ]:
ns = Namespace("http://somewhere#")
vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

print("\n\nShow all the subClassOf relations of the model")#task05

#RDFLib
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
    print(s)
for s1,p1,o1 in g.triples((None, RDFS.subClassOf, s)): #Two levels of subclass!!!! 
  print(s1)

#SPARQL
from rdflib.plugins.sparql import prepareQuery #task4
q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf/rdfs:subClassOf* ns:Person. #Remember "ALL" subclase, not just only the subclass of mainclass
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns}
)


for r in g.query(q1):
  print(r)



Show all the subClassOf relations of the model
http://somewhere#Researcher
http://somewhere#PhDstudent
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [22]:
#RDFLib
#Belong to Person
print("\nShow all individuals of Person")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
#Belong to subClasses
print("\nShow all individuals of subclasses of Person")
for subClasses,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, subClasses)):
    print(s)


#SPARQL //use UNION （symbol ∪）

print("\nShow all individuals SPARQL")
q2 = prepareQuery('''
  SELECT 
    ?Subject 
  WHERE {
    { 
      ?Subject rdf:type ns:Person.
    }
    UNION {
      ?subclasses rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?Subject rdf:type ?subclasses
    } 
  }
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":NS}
)


for r in g.query(q2):
  print(r)




Show all individuals of Person
http://somewhere#SaraJones
http://somewhere#JohnSmith

Show all individuals of subclasses of Person
http://somewhere#JaneSmith

Show all individuals SPARQL
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [23]:
#RDFLib
#Belong to Person
print("\nShow all individuals of Person")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for s1,p1,o1 in g.triples((s,None,None)):
    print(s1,p1,o1)
#Belong to subClasses
print("\nShow all individuals of subclasses of Person")
for subClasses,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, subClasses)):
    for s1,p1,o1 in g.triples((s,None,None)):
      print(s1,p1,o1)


#SPARQL //use UNION （symbol ∪ "or"）

print("\nShow all individuals SPARQL")
q2 = prepareQuery('''
  SELECT 
    ?Subject ?p ?o
  WHERE {
    { 
      ?Subject rdf:type ns:Person.
      ?Subject ?p ?o
    }
    UNION {
      ?subclasses rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?Subject rdf:type ?subclasses.
      ?Subject ?p ?o
    } 
  }
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":NS}
)


for r in g.query(q2):
  print(r)



Show all individuals of Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person

Show all individuals of subclasses of Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vca